## Part 7. Aristotle prover


Harmonic (the company behind Newclid) also has a Lean-based prover called Aristotle. The company provides a web interface and API at https://aristotle.harmonic.fun. This note uses a small, friendly geometry problem to demonstrate the workflow, while the same system is aimed at harder, cutting-edge problems across mathematics (geometry, number theory, analysis, and more). [ARISTOTLE]

Harmonic reports that Aristotle achieved gold medal-level performance on the 2025 International Mathematical Olympiad (IMO), with formally verified solutions. [HN-IMO][BW-IMO] Those IMO announcements are a useful backdrop for this demo: we stay with a simple trapezoid ratio, but the tooling is designed for much harder proofs.

Aristotle has also been used beyond contest problems. In late 2025, it produced a Lean proof of a simplified variant of Erdős Problem #124. Chinese coverage highlighted that the system completed a variant autonomously in about six hours, while other reports noted that the solved statement was easier than the original conjecture. [E124-6H]

A few days later, Harmonic’s tool was credited for helping resolve Erdős Problem #1026. A December 14, 2025 write-up described how the AI tool proved a key step $c(k^2)=1/k$ in Lean as part of the eventual solution. [E1026-48H]

Harmonic’s own news page claims that since Aristotle’s public launch in late 2025, mathematicians have used it to formalize proofs across levels and that the company sees this as a new era of discovery. [HN-MSI]

For broader context on Erdős problems, the community-maintained erdosproblems GitHub database tracks problem status, prizes, formalizations, and links to related resources. It is essentially a living index of the current state of the problem list and related metadata, with an auto-generated table based on the repository's data files. [ERDOS-SOTA]

The accompanying wiki page on AI contributions is more careful and nuanced: it catalogs where AI tools have been used, but also stresses that this is not a benchmark or leaderboard, that literature review can be incomplete, and that assessments are provisional and context-dependent. The page is best read as a reference for tracing claims and understanding the role AI has actually played. [ERDOS-AI]


In [ ]:
import os
import logging
from pathlib import Path
import os
import re

def load_aristotle_api_key():
    key_file = Path.home() / ".aristotle_api_key"
    text = key_file.read_text(encoding="utf-8")

    m = re.search(r'ARISTOTLE_API_KEY="([^"]+)"', text)
    if not m:
        raise RuntimeError("ARISTOTLE_API_KEY not found in ~/.aristotle_api_key")
    return m.group(1)

os.environ["ARISTOTLE_API_KEY"] = load_aristotle_api_key()
print("ARISTOTLE_API_KEY loaded into this kernel")

import aristotlelib
from aristotlelib.project import Project, ProjectInputType

async def prove_polygon_ratio():
    api_key = os.getenv("ARISTOTLE_API_KEY")
    if api_key:
        aristotlelib.set_api_key(api_key)
    else:
        raise RuntimeError("ARISTOTLE_API_KEY environment variable not set")

    logging.basicConfig(level=logging.INFO, format="%(levelname)s - %(message)s")

    problem_text = """
Polygon ratio problem (informal statement).
In trapezoid ABCD, AD ∥ BC, ∠C = 90°, and BC = CD.
Point E lies on CD with DE = AD. Through E draw EF ⟂ AB with F the foot.
Draw BD; it meets AE and EF at G and H, respectively. Prove:
1) BF * HE = GE * BH.
2) BE * DH = BD * EH.
""".strip() + "\n"

    await Project.prove_from_file(
        input_content=problem_text,
        output_file_path="aristotle/polygon_ratio_problem_aristotle.lean",
        validate_lean_project=False,
        project_input_type=ProjectInputType.INFORMAL,
        auto_add_imports=False,
    )

await prove_polygon_ratio()


<div align="center">
<img alt="Aristotle web interface screenshot" src="aristotle_harmonic.fun_output.png" style="height:320px; width:auto; display:block; margin:0 auto;">

*Figure 7.1 The Aristotle web UI at harmonic.fun. The demo job below was queued for a while and then ran for ~20 minutes, suggesting a CPU-heavy run rather than a short GPU burst.*
</div>


### Proof 7.1 Human-readable summary of Aristotle's proof (with detailed derivations)

>
> Aristotle encodes the problem in Lean and outputs a Lean proof file. Below is a human-readable summary of the proof.
>
> Aristotle places the trapezoid in the plane (coordinates in $\mathbb{R}^2$):
>
> $$
> C=(0,0),\; B=(a,0),\; D=(0,a),\; A=(x,a),\; E=(0,a-x).
> $$
>
> These choices encode $AD\parallel BC$, $\angle C=90^\circ$, $BC=CD$, and $DE=AD$ directly. The auxiliary points are defined algebraically:
>
> - $F$ is the orthogonal projection of $E$ onto line $AB$.
> - $G$ is the intersection of $AE$ and $BD$.
> - $H$ is the intersection of $EF$ and $BD$.
>
> From the coordinate definitions, Aristotle derives explicit formulas (in complex coordinates) for the constructed points:
>
> $$
> G=\Big(\frac{x}{2},\; a-\frac{x}{2}\Big).
> $$
>
> Let $AB$ have direction vector $(x-a,\,a)$, so a perpendicular direction is $(a,\,-(x-a))$. The line $EF$ through $E=(0,a-x)$ can be written:
>
> $$
> EF:\quad (X,Y)=(0,a-x)+u\,(a,\,-(x-a)).
> $$
>
> The diagonal $BD$ can be parametrized as:
>
> $$
> BD:\quad (X,Y)=(a,0)+s\,(-a,\,a)=(a(1-s),\,as).
> $$
>
> Solve for the intersection by equating coordinates:
>
> $$
> ua=a(1-s),\quad a-x-u(x-a)=as.
> $$
>
> From $u=1-s$, the second equation gives:
>
> $$
> 2(a-x)=s(2a-x),\quad\Rightarrow\quad s=\frac{2(a-x)}{2a-x},\quad u=\frac{x}{2a-x}.
> $$
>
> Therefore:
>
> $$
> H=\Big(\frac{ax}{2a-x},\; \frac{2a(a-x)}{2a-x}\Big),\qquad (2a\neq x).
> $$
>
> The projection point $F$ is defined using the standard projection formula along $AB$:
>
> $$
> F = B + t(A-B),\quad t=\frac{\langle E-B, A-B\rangle}{\langle A-B, A-B\rangle}.
> $$
>
> If we expand this, the coordinates of $F$ can be written explicitly as:
>
> $$
> F=\Big( a + t(x-a),\; t a \Big),\qquad
>  t=\frac{a^2 + a(x-a)}{(x-a)^2 + a^2}.
> $$
>
> ---
>
> #### Derivation for $BE\cdot DH = BD\cdot EH$
>
> First compute squared lengths:
>
> $$
> BE^2 = (a-0)^2 + (a-x-0)^2 = a^2 + (a-x)^2,
> $$
>
> $$
> BD^2 = (a-0)^2 + (0-a)^2 = 2a^2.
> $$
>
> Now use $H=\big(\frac{ax}{2a-x},\frac{2a(a-x)}{2a-x}\big)$.
>
> For $DH$, subtract $D=(0,a)$:
>
> $$
> DH^2 = \Big(\frac{ax}{2a-x}\Big)^2 + \Big(\frac{2a(a-x)}{2a-x} - a\Big)^2.
> $$
>
> The second component simplifies as:
>
> $$
> \frac{2a(a-x)}{2a-x} - a
> = \frac{2a(a-x) - a(2a-x)}{2a-x}
> = \frac{-ax}{2a-x}.
> $$
>
> So:
>
> $$
> DH^2 = \Big(\frac{ax}{2a-x}\Big)^2 + \Big(\frac{ax}{2a-x}\Big)^2
> = \frac{2a^2x^2}{(2a-x)^2}.
> $$
>
> For $EH$, subtract $E=(0,a-x)$:
>
> $$
> EH^2 = \Big(\frac{ax}{2a-x}\Big)^2 + \Big(\frac{2a(a-x)}{2a-x} - (a-x)\Big)^2.
> $$
>
> The second component simplifies as:
>
> $$
> \frac{2a(a-x)}{2a-x} - (a-x)
> = (a-x)\Big(\frac{2a}{2a-x} - 1\Big)
> = (a-x)\Big(\frac{x}{2a-x}\Big).
> $$
>
> So:
>
> $$
> EH^2 = \frac{a^2x^2}{(2a-x)^2} + \frac{x^2(a-x)^2}{(2a-x)^2}
> = \frac{x^2\big(a^2+(a-x)^2\big)}{(2a-x)^2}.
> $$
>
> Now multiply:
>
> $$
> BE^2\,DH^2 = \big(a^2+(a-x)^2\big)\cdot \frac{2a^2x^2}{(2a-x)^2},
> $$
>
> $$
> BD^2\,EH^2 = 2a^2 \cdot \frac{x^2\big(a^2+(a-x)^2\big)}{(2a-x)^2}.
> $$
>
> These are equal term-by-term, hence:
>
> $$
> BE^2\,DH^2 = BD^2\,EH^2.
> $$
>
> Since all lengths are nonnegative, this implies:
>
> $$
> BE\cdot DH = BD\cdot EH.
> $$
>
> ---
>
> #### Outline for $BF\cdot HE = GE\cdot BH$
>
> The Lean proof follows the same algebraic template:
>
> 1. Express $F,G,H$ in coordinates (with $F$ from the projection formula).
> 2. Convert distances into squared distances using the complex norm.
> 3. Clear denominators (notably $(2a-x)$ and $(x-a)^2+a^2$).
> 4. Reduce the final expression to a polynomial identity in $a$ and $x$.
>
> In Lean, this is reflected by the sequence:
>
> - `unfold` the point definitions,
> - `norm_num` to expand norms and real/imaginary parts,
> - `field_simp` to clear denominators,
> - `ring`/`nlinarith` (and `grind` in the Aristotle output) to finish the algebra.
>
> This is why the proof is algebraically correct even though it is not a geometric (synthetic) proof: the identities become polynomial equalities in $a$ and $x$ once the configuration is coordinatized.


**Note on proof style.** In this problem Aristotle proceeds by coordinates model, so the proof is essentially algebraic and very similar to the essence of what we did in Part 2. However it should be pointed out that in other plane geometry problems it can use more geometric, angle-chasing reasoning. For example, in the Harmonic IMO 2025 P2 formalization [IMO2025P2], the proof structure reflects angle relations and cocyclic arguments rather than pure coordinate algebra. This highlights that Aristotle can deploy higher-level strategies beyond the coordinate approach used here.

Furthermore, as we have discussed at the beginning of the part, Aristotle together with ChatGPT5.2 Pro have become a powerful tools within the methamaticians.


### References

- [ARISTOTLE] Harmonic. "Aristotle" (web interface). Summary: Web interface and API access for the Aristotle prover. URL: https://aristotle.harmonic.fun
- [E124-6H] 36Kr (2025-11-??). "AI solved a 30-year math problem in 6 hours. Terence Tao: ChatGPT and others failed." Summary: Coverage of a simplified Erdős #124 variant solved with Aristotle. URL: https://eu.36kr.com/en/p/3576638922980231
- [E1026-48H] 36Kr (2025-12-14). "Terence Tao is shocked: The “pit” dug by a mathematician in 1975 was filled by AI and netizens around the world in just 48 hours." Summary: Coverage of Erdős #1026 and an Aristotle-assisted Lean step. URL: https://eu.36kr.com/en/p/3596176018898947
- [HN-MSI] Harmonic News (2025). "Harmonic announces mathematician sponsorships to accelerate mathematical superintelligence." Summary: Harmonic’s announcement and claims about Aristotle usage. URL: https://harmonic.fun/news
- [HN-IMO] Harmonic News (2025-07-28). "Aristotle Achieves Gold Medal-Level Performance at the International Mathematical Olympiad". Summary: Harmonic announcement of IMO 2025 results and formal verification. URL: https://harmonic.fun/news
- [BW-IMO] Business Wire (2025-07-28). "Harmonic Announces IMO Gold Medal-Level Performance & Launch of First Mathematical Superintelligence (MSI) AI App." Summary: Press release on IMO 2025 result and Aristotle launch. URL: https://www.businesswire.com/news/home/20250728394917/en/Harmonic-Announces-IMO-Gold-Medal-Level-Performance-Launch-of-First-Mathematical-Superintelligence-MSI-AI-App
- [IMO2025P2] Harmonic AI (2025). "IMO 2025 P2 formalization (Lean)." Summary: Lean formalization file for IMO 2025 Problem 2. URL: https://github.com/harmonic-ai/IMO2025/blob/main/HarmonicLean/IMO2025P2.txt
- [ERDOS-SOTA] Terence Tao et al. (2025). "Erdos Problems" (overview). Summary: Community-maintained status tracker for Erdos problems. URL: https://github.com/teorth/erdosproblems?tab=readme-ov-file
- [ERDOS-AI] Terence Tao et al. (2025). "AI contributions to Erdos problems" (wiki). Summary: Notes on AI involvement in Erdos problem progress. URL: https://github.com/teorth/erdosproblems/wiki/AI-contributions-to-Erd%C5%91s-problems
